In [2]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset
import torch
import torch.nn as nn
import torch.nn.functional as F

# 1. 数据预处理

In [3]:
transform = transforms.Compose([
    transforms.ToTensor(),  
    transforms.Normalize((0.1307,), (0.3081,))  # 标准化处理 将图像转为Tensor (范围从0-255变为0-1)
])

# 2. 加载MNIST数据

In [4]:
train_dataset = datasets.MNIST(
    root='./data',        # 数据存储路径
    train=True,           # 加载训练集
    download=False,        # 自动下载
    transform=transform   # 应用预处理
)

# 3. 提取一部分数据简化训练（前10000个）

In [5]:
subset_indices = list(range(1000))  # 选择前1000个样本
subset_dataset = Subset(train_dataset, subset_indices)

# 4. 加载DataLoader

In [6]:
train_loader = DataLoader(
    subset_dataset,
    batch_size=32,        # 每批加载32个样本
    shuffle=True,         # 训练时打乱数据顺序
    num_workers=1         # 使用1个进程加载数据
)

# 5. 验证数据加载

In [7]:
for batch_idx, (data, target) in enumerate(train_loader):
    print(f"批次 {batch_idx+1}:")
    print(f"  数据形状: {data.shape} (批次大小, 通道数, 高度, 宽度)")
    print(f"  标签形状: {target.shape} (批次大小)")
    print(f"  标签内容: {target.tolist} (num)") 
    
    if batch_idx == 1:  # 只打印前两个批次
        break

批次 1:
  数据形状: torch.Size([32, 1, 28, 28]) (批次大小, 通道数, 高度, 宽度)
  标签形状: torch.Size([32]) (批次大小)
  标签内容: <built-in method tolist of Tensor object at 0x000002DBAAF219A0> (num)
批次 2:
  数据形状: torch.Size([32, 1, 28, 28]) (批次大小, 通道数, 高度, 宽度)
  标签形状: torch.Size([32]) (批次大小)
  标签内容: <built-in method tolist of Tensor object at 0x000002DBAAF23D40> (num)


In [8]:
class MNISTNet(nn.Module):
    def __init__(self):
        super().__init__()  # 必须调用父类构造函数
        
        # 定义网络层结构
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(32 * 7 * 7, 128)  # 注意输入维度的计算
        self.fc2 = nn.Linear(128, 10)          # 10个类别（0-9）
        
    def forward(self, x):
        # 第一层卷积+激活+池化
        x = self.pool(F.relu(self.conv1(x)))  # 输入: [batch, 1, 28, 28] → 输出: [batch, 16, 14, 14]
        
        # 第二层卷积+激活+池化
        x = self.pool(F.relu(self.conv2(x)))  # 输入: [batch, 16, 14, 14] → 输出: [batch, 32, 7, 7]
        
        # 展平为一维向量
        x = x.view(-1, 32 * 7 * 7)  # 展平后的维度: [batch, 32*7*7]
        
        # 全连接层
        x = F.relu(self.fc1(x))     # 输入: [batch, 1568] → 输出: [batch, 128]
        x = self.fc2(x)             # 输入: [batch, 128] → 输出: [batch, 10]
        
        return x

In [9]:
# 实例化模型
model = MNISTNet()
print(model)  # 打印模型结构

MNISTNet(
  (conv1): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=1568, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)
